# Data Preprocessing: Social Media Data

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import warnings

## Setup

In [ ]:
# Set random seed
np.random.seed(1040)

In [ ]:
DATA_DIR = Path("../../data")
input_dir = DATA_DIR / "raw" / "social-media-data"
stream_1_path = input_dir / "Stream1.xlsx"
stream_2_path = input_dir / "Stream2.xlsx"
stream_3_path = input_dir / "Stream3.xlsx"

In [ ]:
# this is ok and will not cause problems
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

stream_1_data = pd.read_excel(stream_2_path, engine="openpyxl") # stream 1 contains the chronologically second part
stream_2_data = pd.read_excel(stream_1_path, engine="openpyxl") # stream 2 contains the chronologically first part
stream_3_data = pd.read_excel(stream_3_path, engine="openpyxl")

data = pd.concat([stream_1_data, stream_2_data, stream_3_data], ignore_index=True);

## First Look

The data contains a great number of different attributes for each observation.
therefore we start by looking at the attributes to get an idea of what to keep and what to get rid of.

First we convert all attribute-names to lowercase and replace white-spaces to underscores to make things simpler

In [ ]:
data.columns = [c.replace(' ', '_').lower() for c in stream_1_data.columns]

In [ ]:
data.info()

Important:

- post_id (ID)
- Sound Bite Text (main text corpus)
- Published Date (GMT+01:00) London (used to create dynamic embeddings)
- Sentiment (used for extrinsic evaluation)

In the following I focus first on these attributes to keep things clear and simple

In [ ]:
important_attributes = ["post_id", "sound_bite_text", "published_date_(gmt+01:00)_london", "sentiment"]

data = pd.DataFrame(data, columns=important_attributes)

For simplicity's sake I chose to rename the attributes to a more readable and manageable form

In [ ]:
data.rename(columns={"source_type": "source", "sound_bite_text":"raw_text", "published_date_(gmt+01:00)_london": "date", "post_id":"id"}, inplace=True)

Let us look at how many values are actually there for the selected attributes

In [ ]:
# number of observations
n = len(data)

# Display relative counts of missing values
data.isnull().sum().divide(n).sort_values(ascending=False)

Both the date and the text have almost no missing values, which is the main thing.
The attribute sentiment will only be used for a part of the evaluation of the embeddings and is therefore not as important.
I therefore decide to go for the following strategy:

Remove observations:

- with missing date
- with missing text

Keep observations:
- with missing sentiment

Let us now describe the key characteristics of our (remaining) data

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

The datatypes are mostly as we would like.
We only convert the date attribute from object to date, since we are working with a time series.

In [ ]:
data['date'].isnull().sum()

Given that temporal word embeddings heavily rely on dates, we consider the date to be crucial. However, out of the 24 tweets available, some lack a date, so I opt to eliminate those observations from the dataset.

In [ ]:
data.dropna(subset=['date'], inplace=True)

In [ ]:
data['date'] =  pd.to_datetime(data['date'])

Since I focus on the temporal change of words, I chose to sort the observations by date because that makes a manual inspection later on more convenient

In [ ]:
data.sort_values('date', inplace=True);

The text of the tweets is the main source of information, lets look how many missing values we encounter here

In [ ]:
null_texts = data['raw_text'].isnull().sum()
empty_texts = data[data['raw_text'].str.len() < 2].count().iloc[0]
print(f"Obsersations with no text: {null_texts}")
print(f"Obsersations with empty text: {empty_texts}")

Since it is only one observation we can safely remove it to prevent it from causing errors later on.

In [ ]:
data.dropna(subset=['raw_text'], inplace=True)

Now lets take a look at the different attributes. Since the task at hand is a sentiment analysis, we focus on this attribute first

In [ ]:
data["sentiment"].unique()

So our target is to predict the sentiment from the text (sound_bite_text).
The sentiment is either:

- Positive
- Negative
- Neutral
- Mixed

In [ ]:
# Get the range of dates
period = (data['date'].min(), data['date'].max())

# Format the output
formatted_range = tuple(date.strftime("%Y-%m-%d") for date in period)
print("Period of time:", formatted_range)

## Text

### 1. Convert to lowercase

In [ ]:
data["text"] = data["raw_text"].str.lower()

# rearrange columns
data = data[['id', 'text', "raw_text", 'date', 'sentiment']]

### 2. Remove Unicode Characters

Eliminate the punctuation, URL, and @

In [ ]:
import re

def clean_text(text):

    # Removes all of them
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

    return text

In [ ]:
data["text"] = data["text"].apply(clean_text)

### 3. Remove Stopwords

In [ ]:
import nltk
# nltk.download('punkt')

def remove_stopwords(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

In [ ]:
data["text"] = data["text"].apply(remove_stopwords)

### 4. Stemming (not used)

Good for extrinsic evaluation but bad for visualization

In [ ]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

def perform_stemming(text):
    stemmer = SnowballStemmer(language = "english")
    word_tokens = word_tokenize(text)
    stemmed_text = [stemmer.stem(word) for word in word_tokens]
    return " ".join(stemmed_text)

**Note: No Stemming is done at the moment**

In [ ]:
# data["text"] = data["text"].apply(perform_stemming)

Check if we inadvertently created some Na, Null Values in our (processed) text column

In [ ]:
data["text"].isna().sum()

In [ ]:
data["text"].isnull().sum()

In [ ]:
def save_to_csv(data, splits: list, sub_dir: str):

    output_dir = DATA_DIR / "processed" / "social-media-data" / sub_dir
    # Create output directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)
    # range of the observations
    lower = data["date"].min()
    upper = data["date"].max()

    for split in splits[1:]:
        split_df = data[(lower <= data['date']) & (data['date'] < split)]
        split_filename = output_dir / f"{lower.strftime('%d_%b')}_to_{split.strftime('%d_%b')}.csv"
        print(f"{lower.strftime('%d_%b')}_to_{split.strftime('%d_%b')}.csv")
        # Save the filtered data to csv, overwrite if exists
        split_df.to_csv(split_filename, index=False, mode='w')
        # Update the lower date for the next iteration
        lower = split

    # take care of second half of the last split
    split_df = data[(lower <= data['date']) & (data['date'] <= upper)]
    split_filename = output_dir / f"{lower.strftime('%d_%b')}_to_{upper.strftime('%d_%b')}.csv"
    print(str(split_filename).split(f"{sub_dir}/")[-1])
    split_df.to_csv(split_filename, index=False, mode='w')

Save whole corpus as one

In [ ]:
output_dir = DATA_DIR / "processed" / "social-media-data"
data.to_csv(output_dir / "corpus_all.csv", index=False, mode='w')

## Splitting

In [ ]:
cleaned_data_path = output_dir / "corpus_all.csv"
data = pd.read_csv(cleaned_data_path)

# Remove text null texts which occurred due to preprocessing and saving to csv
data.dropna(subset=["text"], inplace = True)
# Convert to datetime for the splits
data['date'] =  pd.to_datetime(data['date'])

### Split by events (custom)

To create splits of the data for different time-periods it is sufficient to only run the cells below.
The upper part of the notebook only needs to run once to create the processed_data.csv file

In [ ]:
'''
# List of notable events
griner_release = pd.Timestamp('2022-10-07')
musk_twitter_takeover = pd.Timestamp('2022-10-01')
pelosi_attacked = pd.Timestamp("2022-10-26")
colorado_springs_shooting = pd.Timestamp("2022-11-18")

# did not work
word_cup = pd.Timestamp('2022-11-01')
seoul_halloween = pd.Timestamp('2022-10-28')''';

In [ ]:
# PARAMS TO MODIFY MANUALLY
'''
# sub_dir = "colorado_springs"
splits = [colorado_springs_shooting]
months = [pd.Timestamp("2022-11-18")]

# save_to_csv(data, splits, sub_dir) currently not used''';

### Split in quarters

In [ ]:
sub_dir = "quarter"
quarterly_dates = pd.date_range(start='2022-06-01', end='2023-04-28', freq='QS-JUN')
print(quarterly_dates)

In [ ]:
save_to_csv(data, quarterly_dates, sub_dir)

### Split by months

In [ ]:
sub_dir = "monthly"
# Create a list of first-of-the-month timestamps
first_of_month_dates = pd.date_range(start='2022-06-01', end='2023-04-28', freq='MS')
print(first_of_month_dates)

In [ ]:
save_to_csv(data, first_of_month_dates, sub_dir)

In [ ]:
formatted_range